# Benchmarking `nvjpeg` 📊

In [2]:
%run benchmark_nvjpeg_LoLa.py

Combination 1/168 - 1
Combination 2/168 - 1
Combination 3/168 - 1
Combination 4/168 - 1
Combination 5/168 - 1
Combination 6/168 - 1
Combination 7/168 - 1
Combination 8/168 - 1
Combination 9/168 - 1
Combination 10/168 - 1
Combination 11/168 - 1
Combination 12/168 - 1
Combination 13/168 - 1
Combination 14/168 - 1
Combination 15/168 - 1
Combination 16/168 - 1
Combination 17/168 - 1
Combination 18/168 - 1
Combination 19/168 - 1
Combination 20/168 - 1
Combination 21/168 - 1
Combination 22/168 - 1
Combination 23/168 - 1
Combination 24/168 - 1
Combination 25/168 - 1
Combination 26/168 - 1
Combination 27/168 - 1
Combination 28/168 - 1
Combination 29/168 - 1
Combination 30/168 - 1
Combination 31/168 - 1
Combination 32/168 - 1
Combination 33/168 - 1
Combination 34/168 - 1
Combination 35/168 - 1
Combination 36/168 - 1
Combination 37/168 - 1
Combination 38/168 - 1
Combination 39/168 - 1
Combination 40/168 - 1
Combination 41/168 - 1
Combination 42/168 - 1
Combination 43/168 - 1
Combination 44/168 -

KeyboardInterrupt: 

# 📈 Data Visualization & Analysis

Now we will plot the collected data to understand the results. We will focus on:
1.  **Rate-Distortion:** How do quality metrics (like VMAF) change with file size?
2.  **Quality vs. File Size:** How does the `-quality` setting impact file size for different subsampling methods?
3.  **Encoding Performance:** Which Format method is fastest? How does resolution affect encoding time?
4.  **Decoding Performance:** How does resolution affect decoding time?

In [ ]:
# --- Main Configuration ---
# Resolutions to test (Name: (Width, Height))
RESOLUTIONS = {
    '1280x720': (1280, 720),
    '1920x1080': (1920, 1080),
    '3840x2160': (3840, 2160),
}

RESOLUTION_TO_FILENAME = {
    '1280x720': 'frame_hd.rgb',
    '1920x1080': 'frame_fullhd.rgb',
}

# Parameters to benchmark
QUALITIES = range(40, 96, 1)
SUBSAMPLINGS = ['4:2:0']

SUBSAMPLING_TO_PARAM = {
    '4:2:0': '420'
}

In [ ]:
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('results_nvjpeg_LoLa.csv')
df.head()

In [ ]:
# Set plot style
sns.set_theme(style="whitegrid")

## Encoding Parameters vs. quality metrics

In [ ]:
x = 'Quality'
y = 'PSNR'

g = sns.relplot(
    data=df,
    x=x,
    y=y,
    #hue='Frame',
    col='Resolution',
    kind='line',
    height=5,
    aspect=1.2,
    col_order=RESOLUTIONS.keys(),
    facet_kws={'sharex': False}
)

g.figure.suptitle(f'{x} vs. {y}', y=1.03, fontsize=16)
g.set_axis_labels(x, y)
plt.show()

x = 'Quality'
y = 'SSIM'

g = sns.relplot(
    data=df,
    x=x,
    y=y,
    #hue='Frame',
    col='Resolution',
    kind='line',
    height=5,
    aspect=1.2,
    col_order=RESOLUTIONS.keys(),
    facet_kws={'sharex': False}
)

g.figure.suptitle(f'{x} vs. {y}', y=1.03, fontsize=16)
g.set_axis_labels(x, y)
plt.show()

## Encoding Parameters vs. File Size


In [ ]:
g = sns.relplot(
    data=df,
    x='Quality',
    y='Compressed Size (KB)',
    #hue='Frame',
    col='Resolution',
    kind='line',
    height=5,
    aspect=1.2,
    col_order=RESOLUTIONS.keys(),
    facet_kws={'sharey': False}
)
g.figure.suptitle('Quality vs Compressed File Size', y=1.03, fontsize=16)
g.set_axis_labels('JPEG Quality', 'Compressed Size (KB)')
plt.show()

fps = 30
g = sns.relplot(
    data=df,
    x='Quality',
    y=df['Compressed Size (KB)'] * fps / 1000,
    #hue='Frame',
    col='Resolution',
    kind='line',
    height=5,
    aspect=1.2,
    col_order=RESOLUTIONS.keys(),
    facet_kws={'sharey': False},
)
g.figure.suptitle(f'Quality vs Compressed Bitrate @{fps}fps', y=1.03, fontsize=16)
g.set_axis_labels('JPEG Quality', f'Bitrate@{fps}fps (MB/s)')
plt.show()

We can observe an explosion in file size from quality 95 onwards, and a sharp increase from quality 90 onwards.

## Encoding Parameters vs. Performance

In [ ]:
g = sns.relplot(
    data=df,
    x='Quality',
    y=df['Avg Encode Time (s)'] * 1000,
    #hue='Frame',
    kind='line',
    col='Resolution',
    col_order=RESOLUTIONS.keys(),
    height=5,
    aspect=1.2,
    facet_kws={'sharey': False}
)
g.figure.suptitle('Quality vs Encode time', y=1.03, fontsize=16)
g.set_axis_labels('JPEG Quality', 'Encode time (milliseconds)')
plt.show()

g = sns.relplot(
    data=df,
    x='Quality',
    y=df['Avg Decode Time (s)'] * 1000,
    #hue='Frame',
    kind='line',
    col='Resolution',
    col_order=RESOLUTIONS.keys(),
    height=5,
    aspect=1.2,
    facet_kws={'sharey': False}
)
g.figure.suptitle('Quality vs Decode time', y=1.03, fontsize=16)
g.set_axis_labels('JPEG Quality', 'Decode time (milliseconds)')
df = df[df['Avg Encode Time (s)'] * 1000 + df['Avg Decode Time (s)'] * 1000 <= 2 * 2]
plt.show()

### Perform average avross frames

In [ ]:
df_agg = df.groupby(['Quality', 'Resolution']).agg({
    #'Quality': 'first',  # kept as-is (assuming same value across group)
    'Avg Encode Time (s)': 'mean',  # averaged
    'Avg Decode Time (s)': 'mean',
    'Compressed Size (KB)': 'mean',
    'PSNR': 'mean',
    'SSIM': 'mean',
}).reset_index()
df_agg

In [ ]:
df_ultrahd = df_agg[df_agg['Resolution'] == '3840x2160']
df_fullhd = df_agg[df_agg['Resolution'] == '1920x1080']
df_hd = df_agg[df_agg['Resolution'] == '1280x720']

### UltraHD

In [ ]:
df_ultrahd[df_ultrahd['Avg Encode Time (s)'] * 1000 + df_ultrahd['Avg Decode Time (s)'] * 1000 <= 2 * 2].sort_values(by='SSIM', ascending=False).head(5)
#df = df[df['Avg Encode Time (s)'] * 1000 <= 2]
#df = df[df['Avg Decode Time (s)'] * 1000 <= 2]
#df.sort_values(by='SSIM', ascending=False).head(5)

In [ ]:
df_ultrahd[df_ultrahd['Avg Encode Time (s)'] * 1000 + df_ultrahd['Avg Decode Time (s)'] * 1000 <= 1.81 * 2].sort_values(by='SSIM', ascending=False).head(5)
#df = df[df['Avg Encode Time (s)'] * 1000 <= 1.5]
#df = df[df['Avg Decode Time (s)'] * 1000 <= 1.5]
#df.sort_values(by='SSIM', ascending=False).head(5)

### FullHD

In [ ]:
df_fullhd[df_fullhd['Avg Encode Time (s)'] * 1000 + df_fullhd['Avg Decode Time (s)'] * 1000 <= 2 * 2].sort_values(by='SSIM', ascending=False).head(5)


In [ ]:
df_fullhd[df_fullhd['Avg Encode Time (s)'] * 1000 + df_fullhd['Avg Decode Time (s)'] * 1000 <= 1.5 * 2].sort_values(by='SSIM', ascending=False).head(5)

In [ ]:
df_fullhd[df_fullhd['Avg Encode Time (s)'] * 1000 + df_fullhd['Avg Decode Time (s)'] * 1000 <= 1 * 2].sort_values(by='SSIM', ascending=False).head(5)


In [ ]:
df_fullhd[df_fullhd['Avg Encode Time (s)'] * 1000 + df_fullhd['Avg Decode Time (s)'] * 1000 <= 0.80 * 2].sort_values(by='SSIM', ascending=False).head(5)

### HD

In [ ]:
df_hd[df_hd['Avg Encode Time (s)'] * 1000 + df_hd['Avg Decode Time (s)'] * 1000 <= 1 * 2].sort_values(by='SSIM', ascending=False).head(5)


In [ ]:
df_hd[df_hd['Avg Encode Time (s)'] * 1000 + df_hd['Avg Decode Time (s)'] * 1000 <= 0.5 * 2].sort_values(by='SSIM', ascending=False).head(5)


In [ ]:
df_hd[df_hd['Avg Encode Time (s)'] * 1000 + df_hd['Avg Decode Time (s)'] * 1000 <= 0.4 * 2].sort_values(by='SSIM', ascending=False).head(5)
